In [2]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA as sklearnPCA
import pandas as pd

from pyclustering.cluster.dbscan import dbscan
from pyclustering.cluster import cluster_visualizer_multidim
from pyclustering.utils import read_sample

import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

## Load data

In [3]:
X = np.load('X2.npy')
print(len(X))

3183


## Apply DBSCAN

In [4]:
min_s = 1
eps = 4.7
clustering = DBSCAN(eps=eps, min_samples=min_s).fit(X)

l = []
cl = clustering.labels_
for element in cl:
    if element not in l:
        l.append(element)
print(' nr_clusters=' + str(len(l)) )

 nr_clusters=23


In [5]:
aparitii = np.zeros([len(l)])
for elem in cl:
        aparitii[elem]+=1

In [6]:
print(len(aparitii))
for i in range(0,len(aparitii)):
    print(i, aparitii[i])

23
0 3161.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
21 1.0
22 1.0


## Apply DBSCAN V2

In [55]:
# Sample for cluster analysis.
sample = X

eps = 3.7
min_s = 1
dbscan_instance = dbscan(sample, eps, min_s)
dbscan_instance.process()
clusters = dbscan_instance.get_clusters()
noise = dbscan_instance.get_noise()

print(len(clusters))
print(len(noise))

13
217


In [56]:
labels = np.zeros(3183)
i = 1
for cls in clusters:
    print(len(cls))
    for elem in cls:
        labels[elem] = i
        
    i += 1

2942
2
2
2
2
2
2
2
2
2
2
2
2


## Apply PCA

In [7]:
pca = sklearnPCA(n_components=30) 
pca_df = pd.DataFrame(pca.fit_transform(X))
df_pca = pca_df.values

## Write metadata

In [9]:
with open("./log-47-1/cl.tsv", "w") as f:
    for elem in cl:
        print(str(elem), file=f)

## For Tensorboard

In [10]:
PATH = os.getcwd()

LOG_DIR = PATH + './log-47-1/'

metadata = os.path.join(LOG_DIR, 'cl.tsv')

tf_data = tf.Variable(df_pca)

with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name
    
    embedding.metadata_path = metadata
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
len(clusters)

3

3167